In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## EDA for articles.csv

In [ ]:
articles_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv")

In [ ]:
articles_df.head()

In [ ]:
articles_df.describe()

In [ ]:
len(articles_df.columns)

In [ ]:
# iterating the columns
for col in articles_df.columns:
    print(col)

### Describe for each column

In [ ]:
len(articles_df['article_id'].unique())

In [ ]:
len(articles_df['product_code'].unique())

In [ ]:
temp_df = articles_df.groupby(['product_code']).size().reset_index(name='counts')

In [ ]:
len(temp_df[temp_df['counts']>0]['product_code'].unique())

#### Remove Uncessary Columns

In [ ]:
reduced_article_df = articles_df[['article_id', 'product_code', 'prod_name', 'product_type_no', 'product_group_name', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'index_code', 'index_group_no', 'section_no', 'garment_group_no', 'detail_desc']]

In [ ]:
reduced_article_df.head()

In [ ]:
articles_df[['perceived_colour_value_name', 'perceived_colour_master_name', 'colour_group_name']].head()

In [ ]:
len(articles_df['product_code'].unique())

## EDA for sample_submission.csv

In [ ]:
sample_submission_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv")
sample_submission_df.head()

In [ ]:
sample_submission_df.describe()

## EDA for transactions_train.csv

In [ ]:
transactions_train_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")
transactions_train_df.head()

In [ ]:
transactions_train_df['t_dat'].unique()

#### Convert to datetime column

In [ ]:
transactions_train_df['t_dat'] = pd.to_datetime(transactions_train_df['t_dat'])

In [ ]:
df_filtered = transactions_train_df[transactions_train_df['t_dat'].dt.strftime('%Y') == '2020']

In [ ]:
df_filtered.to_csv("2020_data.csv")

In [ ]:
len(df_filtered)

### Drop uncessary columns

In [ ]:
df_filtered.drop(['t_dat', 'price', 'sales_channel_id'], axis=1, inplace=True)

In [ ]:
merged_df = pd.merge(reduced_article_df, df_filtered, how='right', on = 'article_id')

In [ ]:
len(merged_df)

In [ ]:
len(transactions_train_df)

In [ ]:
transactions_train_df.describe()

In [ ]:
merged_df.head()

#### Check Duplicated Value Exists

In [ ]:
merged_df.duplicated()

#### Drop Duplicated Values

In [ ]:
merged_df = merged_df.drop_duplicates()

In [ ]:
len(merged_df)

## EDA for customers.csv

In [ ]:
customer_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv")
customer_df.head()

In [ ]:
customer_df.head()